# [ACCOUNTING] post-dbNSFP filtering and local download

## confirming that non-detected & detected CK positions are correct/ready to be made into .bed file for VEP processing!

12.10.19

### Brief reminder of dbNSFP filtering from searching old html files in 'MAPPING/HTML_before_clean/': 
- only kept rows where stable canonical uniprot ID was found (searched 3903...2 were chr multimapping so total 3901)
    - 2 uniprot IDs were mapping to multiple chromosomes (error in bed file from uniprot)
- lost rows that did not have valid hg19 coordinates, failed CADDhg19 annotation (2165 rows)
- P42166 & P42167 are isoforms on chr12 that caused issues in dbnsfp, ID was combined for some rows because target position found in both IDs..i think i made combo rows into 'P42166'
- BUG FIX entailed searching for canonical uniprot ID in dbNSFP col list, then using index, comparing AApos in list based on index
    - result was equivalent AA position for found canonical uniprot ID!
- final cleaning of merged dbNSFP (all chr merge), filtered out non missense based on aagain column like 'x', also AA positions with negative (i.e. C-1) splice variants

## in K markdown:
1. check that all uniprot IDs between detected and non detected files match
2. check that all targets are found in subset of IDs
3. check that all nontarget CK positions are accounted for by comparing to uniprot fasta raw CK totals
4. check that all C and K codons have 7 nucleotide changes overlapping...allowed 2 positions to not have 7, both are nontarget positions..notes below. 
5. make bed files

```python
# from non-detected file, more detail at end of markdown

    pos_ID	codon_changes(7)
    Q86W56_K115	  5
    Q9Y6J0_C571	  6
    
```

### FINAL ACCOUNTING of filtered/checked CK positions for .bed file:
* total uniprot IDs = 3840
* all targets found with 7 changes per codon
* only 2 CK non-targets missing and 2 aminos with incorrect codon change count

| TYPE | Cysteine | Lysine | 
|-------|--------------|-----------|
| Low | 895 | 3401 | 
| Medium | 412 | 660 |
| High | 94 | 302 |
| Panreactive | 5088 | 7804 |
| TargetSM | 708 | 115 |
| EVER-labeled | 6057 | 8868 |
| not-labeled | 34050 | 140652 |

# [1] check that detected and detected files with coordinates of Cysteine and Lysine have the same unique uniprot IDs

In [7]:
# packages
import os
import sys
import numpy as np
import pandas as pd
from statistics import stdev 
from ast import literal_eval # for mismap_score func
from statistics import mean

In [8]:
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000
display(HTML("<style>.container {width:90% !important;}</style>"))
sys.path.append("/Users/mariapalafox/Desktop/Toolbox")
from all_funx import *

In [9]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/BUG_FIX")

In [10]:
det = pd.read_csv("detected_appeneded_merge.csv", low_memory=False)
notd = pd.read_csv("not_detected_appended.csv", low_memory=False)

In [13]:
det.head(3)

,pos_id19,pos_id38_x,aaref_x,aaalt_x,matched_aapos_x,CADD_phred_hg38,matched_UKBID_x,pos_dict,matched_target,CADD_phred_hg19,CADDdiff_38minus19,Amino_acids,pos_ID,Cys_reactivity,Cys_react_threshold,Cys_target_label,Lys_reactivity,Lys_react_threshold,Lys_target_label,#chr,pos(1-based),ref,alt,aaref_y,aaalt_y,rs_dbSNP151,hg19_chr,hg19_pos(1-based),hg18_chr,hg18_pos(1-based),aapos,genename,Ensembl_geneid,Ensembl_transcriptid,Ensembl_proteinid,Uniprot_acc,Uniprot_entry,HGVSc_ANNOVAR,HGVSp_ANNOVAR,HGVSc_snpEff,HGVSp_snpEff,HGVSc_VEP,HGVSp_VEP,APPRIS,GENCODE_basic,TSL,VEP_canonical,cds_strand,refcodon,codonpos,codon_degeneracy,Ancestral_allele,AltaiNeandertal,Denisova,VindijiaNeandertal,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,LRT_Omega,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationTaster_model,MutationTaster_AAE,MutationAssessor_score,MutationAssessor_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,VEST4_score,VEST4_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,Reliability_index,M-CAP_score,M-CAP_rankscore,M-CAP_pred,REVEL_score,REVEL_rankscore,MutPred_score,MutPred_rankscore,MutPred_protID,MutPred_AAchange,MutPred_Top5features,MVP_score,MVP_rankscore,MPC_score,MPC_rankscore,PrimateAI_score,PrimateAI_rankscore,PrimateAI_pred,DEOGEN2_score,DEOGEN2_rankscore,DEOGEN2_pred,Aloft_Fraction_transcripts_affected,Aloft_prob_Tolerant,Aloft_prob_Recessive,Aloft_prob_Dominant,Aloft_pred,Aloft_Confidence,CADD_raw,CADD_raw_rankscore,CADD_phred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,fathmm-MKL_coding_group,fathmm-XF_coding_score,fathmm-XF_coding_rankscore,fathmm-XF_coding_pred,Eigen-raw_coding,Eigen-raw_coding_rankscore,Eigen-pred_coding,Eigen-PC-raw_coding,Eigen-PC-raw_coding_rankscore,Eigen-PC-phred_coding,GenoCanyon_score,GenoCanyon_rankscore,integrated_fitCons_score,integrated_fitCons_rankscore,integrated_confidence_value,GM12878_fitCons_score,GM12878_fitCons_rankscore,GM12878_confidence_value,H1-hESC_fitCons_score,H1-hESC_fitCons_rankscore,H1-hESC_confidence_value,HUVEC_fitCons_score,HUVEC_fitCons_rankscore,HUVEC_confidence_value,LINSIGHT,LINSIGHT_rankscore,GERP++_NR,GERP++_RS,GERP++_RS_rankscore,phyloP100way_vertebrate,phyloP100way_vertebrate_rankscore,phyloP30way_mammalian,phyloP30way_mammalian_rankscore,phyloP17way_primate,phyloP17way_primate_rankscore,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons30way_mammalian,phastCons30way_mammalian_rankscore,phastCons17way_primate,phastCons17way_primate_rankscore,SiPhy_29way_pi,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,bStatistic,bStatistic_rankscore,1000Gp3_AC,1000Gp3_AF,1000Gp3_AFR_AC,1000Gp3_AFR_AF,1000Gp3_EUR_AC,1000Gp3_EUR_AF,1000Gp3_AMR_AC,1000Gp3_AMR_AF,1000Gp3_EAS_AC,1000Gp3_EAS_AF,1000Gp3_SAS_AC,1000Gp3_SAS_AF,TWINSUK_AC,TWINSUK_AF,ALSPAC_AC,ALSPAC_AF,UK10K_AC,UK10K_AF,ESP6500_AA_AC,ESP6500_AA_AF,ESP6500_EA_AC,ESP6500_EA_AF,ExAC_AC,ExAC_AF,ExAC_Adj_AC,ExAC_Adj_AF,ExAC_AFR_AC,ExAC_AFR_AF,ExAC_AMR_AC,ExAC_AMR_AF,ExAC_EAS_AC,ExAC_EAS_AF,ExAC_FIN_AC,ExAC_FIN_AF,ExAC_NFE_AC,ExAC_NFE_AF,ExAC_SAS_AC,ExAC_SAS_AF,ExAC_nonTCGA_AC,ExAC_nonTCGA_AF,ExAC_nonTCGA_Adj_AC,ExAC_nonTCGA_Adj_AF,ExAC_nonTCGA_AFR_AC,ExAC_nonTCGA_AFR_AF,ExAC_nonTCGA_AMR_AC,ExAC_nonTCGA_AMR_AF,ExAC_nonTCGA_EAS_AC,ExAC_nonTCGA_EAS_AF,ExAC_nonTCGA_FIN_AC,ExAC_nonTCGA_FIN_AF,ExAC_nonTCGA_NFE_AC,ExAC_nonTCGA_NFE_AF,ExAC_nonTCGA_SAS_AC,ExAC_nonTCGA_SAS_AF,ExAC_nonpsych_AC,ExAC_nonpsych_AF,ExAC_nonpsych_Adj_AC,ExAC_nonpsych_Adj_AF,ExAC_nonpsych_AFR_AC,ExAC_nonpsych_AFR_AF,ExAC_nonpsych_AMR_AC,ExAC_nonpsych_AMR_AF,ExAC_nonpsych_EAS_AC,ExA

In [15]:
notd.head(3)

,pos_id19,pos_id38,aaref,aaalt,matched_aapos,HGVSp_VEP,CADD_phred_hg38,matched_UKBID,pos_dict,matched_target,CADD_phred_hg19,CADDdiff_38minus19,lost_amino,gained_amino,Amino_acids,pos_ID_falseCKtarget
0,10_000093156_C_A,10_000047216_C_A,K,N,392,p.Lys358Asn;p.Lys355Asn;p.Lys392Asn,23.0,Q3ZCM7,"{12: 'C', 239: 'C', 303: 'C', 354: 'C'}",False,22.9,0.1,Lys,Asn,Lys/Asn,Q3ZCM7_K392
1,10_000093156_C_G,10_000047216_C_G,K,N,392,p.Lys358Asn;p.Lys355Asn;p.Lys392Asn,23.1,Q3ZCM7,"{12: 'C', 239: 'C', 303: 'C', 354: 'C'}",False,22.8,0.3,Lys,Asn,Lys/Asn,Q3ZCM7_K392
2,10_000093157_T_A,10_000047217_T_A,K,M,392,p.Lys358Met;p.Lys355Met;p.Lys392Met,23.2,Q3ZCM7,"{12: 'C', 239: 'C', 303: 'C', 354: 'C'}",False,23.2,0.0,Lys,Met,Lys/Met,Q3ZCM7_K392


In [35]:
# DETECTED FILE
# how many unique uniprot IDs: 
uniqueCount(det, 'matched_UKBID_x')
# how many unique c and k positions
uniqueCount(det, 'pos_ID')

# not detected file
uniqueCount(notd, 'matched_UKBID')
uniqueCount(notd, 'pos_ID_falseCKtarget')

len of col:  106064
len of col set:  3880

len of col:  106064
len of col set:  15152

len of col:  1259801
len of col set:  3878

len of col:  1259801
len of col set:  179957



In [41]:
# controlling for same uniprot IDs in both detected and non detected files
detID = list(det.matched_UKBID_x)
notdID = list(notd.matched_UKBID)

difflist = diffList(detID, notdID)
intersec = intersectionCheck(detID, notdID)

print(difflist)
print(len(intersec))
# result of detectecd - notdetected, so these IDs are in the det set only

'''
['Q9Y4H4', 'P63220', 'O95777', 'P62857']
3876
'''
intersec = list(intersec)
print(len(intersec))

different number of IDs :  4
['Q9Y4H4', 'P63220', 'O95777', 'P62857']
3876
3876


In [52]:
# combining det and not det CK positions and entries to compare to uniprot fasta file
sim_det = det[['matched_UKBID_x', 'pos_ID']]
sim_not = notd[['matched_UKBID', 'pos_ID_falseCKtarget']]
# rename col
sim_det.columns = ['entry', 'pos_ID']
sim_not.columns = ['entry', 'pos_ID']
# combine 
dbnsfp_ck = pd.concat([sim_det, sim_not])
dbnsfp_ck.drop_duplicates(inplace=True)

# drop rows with IDs not in shared 3876 set
dbnsfp_3876 = addcolumnconditionalDrop(intersec, dbnsfp_ck, 'entry', 'shared3876')
detected3876 = addcolumnconditionalDrop(intersec, sim_det, 'entry', 'shared3876')

  shared3876   Count
0       True  195046
1      False      63
dropping false
df shape post drop:  (195046, 3)
  shared3876   Count
0       True  105959
1      False     105
dropping false
df shape post drop:  (105959, 3)
(195046, 3)
(105959, 3)


/Users/mariapalafox/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/mariapalafox/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/mariapalafox/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

In [56]:
print(len(detected3876.entry.unique()))
detected3876.drop_duplicates(inplace=True)
print(detected3876.head(5))
# ONLY DETECTED POSITIONS IN DBNSFP

3876
     entry       pos_ID shared3876
0   Q3ZCM7  Q3ZCM7_C354       True
7   Q3ZCM7  Q3ZCM7_C303       True
14  Q3ZCM7  Q3ZCM7_C239       True
21  Q3ZCM7   Q3ZCM7_C12       True
28  Q92615  Q92615_C633       True


/Users/mariapalafox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [75]:
# grouping dbnsfp detected file only by uniprot ID
detected2 = detected3876.groupby('entry')['pos_ID'].apply(lambda x: len(x))

In [76]:
# from chemoproteomics data processing:
targetcounts = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/REFERENCE_2018_mappedCK/SUBSETS/detection_freq_3953_CK_all.csv")
c_target = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/REFERENCE_2018_mappedCK/SUBSETS/Cys2018_ever_labeled_6234.csv")
k_target = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/REFERENCE_2018_mappedCK/SUBSETS/Lys2018_ever_labeled_9042.csv")

In [77]:
detected2 = pd.DataFrame(detected2)
detected2.reset_index(inplace=True)
detected2.columns = ['ID', 'dbNSFP_target_count']
detected2.set_index('ID', inplace=True)
targetcounts.sort_values('ID', inplace=True)
targetcounts.set_index('ID', inplace=True)

,dbNSFP_target_count
ID,
A0AVT1,11
A0JNW5,1
A0MZ66,1
A1KXE4,1
A1L0T0,1
...,...
Q9Y6M9,1
Q9Y6W3,2
Q9Y6W5,1


In [67]:
detected2.reset_index(inplace=True)
targetcounts.reset_index(inplace=True)
mergetargets = pd.merge(targetcounts,detected2,how='inner',on=['ID'])
mergetargets

,ID,times_detected,count_C_targets,count_K_targets,dbNSFP_target_count
0,A0AVT1,11,6,5,11
1,A0JNW5,1,0,1,1
2,A0MZ66,1,1,0,1
3,A1KXE4,1,1,0,1
4,A1L0T0,1,0,1,1
...,...,...,...,...,...
3837,Q9Y6M9,1,1,0,1
3838,Q9Y6W3,2,2,0,2
3839,Q9Y6W5,1,1,0,1
3840,Q9Y6W6,1,1,0,1


In [68]:
mergetargets['check_target_total'] = mergetargets['times_detected'] - mergetargets['dbNSFP_target_count']
checkColumnValues(mergetargets, 'check_target_total')

   check_target_total  Count
0                   0   3841
1                   1      1


In [70]:
# how many IDs are in dbNSFP detected file and not in newly filtered step 1 of mapping data (post 2012 positions mapping)
targetlist = targetcounts.ID.tolist()
detectedlist = detected2.ID.tolist()
# checking list of IDs not found in chemoproteomic filtered files
difflist = diffList(detectedlist, targetlist)
difflist

different number of IDs :  34


['P54136',
 'P86791',
 'P68371',
 'Q71DI3',
 'Q9Y2S7',
 'O75044',
 'P58107',
 'Q9NXV6',
 'Q99873',
 'P63261',
 'P62826',
 'Q8WUH1',
 'Q4G0P3',
 'P0DMV9',
 'Q8WUH6',
 'P82921',
 'P19105',
 'P06753',
 'Q8IYB9',
 'P04406',
 'Q9BTY7',
 'P68363',
 'Q9BQE3',
 'P80303',
 'Q9GZM3',
 'Q99613',
 'P60174',
 'Q96TA1',
 'P08238',
 'Q86YV5',
 'P04350',
 'P26640',
 'Q96GY3',
 'P63241']

## why are some uniprot IDs in the dbNSFP detected file and not in uniprot chemoproteomics ever labeled files??

**these uniprot IDs were dropped when i corrected the CK position checked script for 2012 chemoposition mapping to 2018 reference sequences**
- only keep IDs where all detected positions found (n=3841)...final count 3840

# [2] check that detected and detected files have all C and K positions for uniprot ID 

In [74]:
all_targets = mergetargets[mergetargets['check_target_total'] != 1].copy()
all_targets

,ID,times_detected,count_C_targets,count_K_targets,dbNSFP_target_count,check_target_total
0,A0AVT1,11,6,5,11,0
1,A0JNW5,1,0,1,1,0
2,A0MZ66,1,1,0,1,0
3,A1KXE4,1,1,0,1,0
4,A1L0T0,1,0,1,1,0
...,...,...,...,...,...,...
3837,Q9Y6M9,1,1,0,1,0
3838,Q9Y6W3,2,2,0,2,0
3839,Q9Y6W5,1,1,0,1,0
3840,Q9Y6W6,1,1,0,1,0


In [84]:
all_targets.sum()

ID                     A0AVT1A0JNW5A0MZ66A1KXE4A1L0T0A2A288A2RUC4A3KN...
times_detected                                                     14926
count_C_targets                                                     6058
count_K_targets                                                     8868
dbNSFP_target_count                                                14926
check_target_total                                                     0
dtype: object

In [118]:
# dropped id found with weird K codon missense counts and checking again post drop
# drop rows with IDs not in shared 3876 set
all3840 = all_targets.ID.tolist()
all3840.remove('Q8WZ42')
print(len(all3840))

det_final = addcolumnconditionalDrop(all3840, det, 'matched_UKBID_x', 'shared3841')
not_final = addcolumnconditionalDrop(all3840, notd, 'matched_UKBID', 'shared3841')

  shared3841   Count
0       True  104475
1      False       7
dropping false
df shape post drop:  (104475, 400)
  shared3841    Count
0       True  1222911
1      False    24297
dropping false
df shape post drop:  (1222911, 17)
len of col:  104475
len of col set:  3840

len of col:  104475
len of col set:  14925

len of col:  1222911
len of col set:  3840

len of col:  1222911
len of col set:  174702



In [117]:
all_targets.to_csv("all_targetCK_found_in_UKBIDs3840_dbNSFP.csv")

## [3] checking that all missense for C and K codon have 7 reported alterations
- all target positions had 7 changes for codon (correct)
- non targets had 3 IDs with not 7 count for codon changes, dropped worst ID

In [119]:
# checking that all missense and in file
det_final_group = det_final.groupby('pos_ID')['pos_id19'].apply(lambda x:len(x))
not_final_group = not_final.groupby('pos_ID_falseCKtarget')['pos_id19'].apply(lambda x:len(x))

det_final_group = pd.DataFrame(det_final_group)
checkColumnValues(det_final_group, 'pos_id19') # all target codons had 7

not_final_group = pd.DataFrame(not_final_group)
checkColumnValues(not_final_group, 'pos_id19')
'''
BEFORE DROPPING TTN gene
pos_id19   Count
0         7  178139
1        14      16
2         6       1
3         5       1
# detected file has 3 uniprot IDs with amino acids containing incorrect missense codon count
'''

   pos_id19  Count
0         7  14925
   pos_id19   Count
0         7  174700
1         6       1
2         5       1


'\nBEFORE DROPPING TTN gene\npos_id19   Count\n0         7  178139\n1        14      16\n2         6       1\n3         5       1\n# detected file has 3 uniprot IDs with amino acids containing incorrect missense codon count\n'

In [124]:
# BEFORE DROPPING Q8WZ42 ID
#INCORRECT CODON NUMBER IN NOT detected file from dbNSFP
not_final_group[not_final_group['pos_id19'] != 7]

,pos_id19
pos_ID_falseCKtarget,
Q86W56_K115,5
Q9Y6J0_C571,6


## keeping: 
Q9Y6J0 has 2 targets is Calcineurin-binding protein cabin-1 important for T cell receptor signaling

Q86W56 has 3 target is PolyADP-ribose glycohydrolase with PCNA binding domain dont delete!

## dropping: 
Q8WZ42 has 1 target TTN important for straited muscle

```
pos_ID_falseCKtarget	
Q8WZ42_K11429	14
Q8WZ42_K11435	14
Q8WZ42_K11436	14
Q8WZ42_K11443	14
Q8WZ42_K11444	14
Q8WZ42_K11451	14
Q8WZ42_K11457	14
Q8WZ42_K11463	14
Q8WZ42_K11464	14
Q8WZ42_K11471	14
Q8WZ42_K11472	14
Q8WZ42_K11479	14
Q8WZ42_K11485	14
Q8WZ42_K11491	14
Q8WZ42_K11492	14
Q8WZ42_K11500	14
```

# ALL targets in 3840 uniprot IDs found!

In [127]:
# DETECTED FILE 
uniqueCount(det_final, 'matched_UKBID_x')
uniqueCount(det_final, 'pos_ID')
print()
# not detected file
uniqueCount(not_final, 'matched_UKBID')
uniqueCount(not_final, 'pos_ID_falseCKtarget')

len of col:  104475
len of col set:  3840

len of col:  104475
len of col set:  14925


len of col:  1222911
len of col set:  3840

len of col:  1222911
len of col set:  174702



# [4] now checking CK total counts for these IDs (uniprot fasta) vs total CK not detected from dbNSFP
- all CK found exceptp 2 postions, great!

In [134]:
ukb = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/REFERENCE_2018_mappedCK/SUBSETS/countsfromfastaCCDS_filter_counts_all_AA.csv")
ukb_final = addcolumnconditionalDrop(all3840, ukb, 'ID', 'shared3840')
ukb_final.sum()

  shared3840  Count
0      False  14592
1       True   3840
dropping false
df shape post drop:  (3840, 30)


Unnamed: 0                                              33973922
ID             Q9HAS0Q86X76Q9NQR4Q9Y314Q04721Q9UHY1P49116Q166...
entryName      NJMU_HUMANNIT1_HUMANNIT2_HUMANNOSIP_HUMANNOTC2...
Length                                                   2276173
proSequence    MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGS...
A                                                         164566
B                                                              0
C                                                          40115
D                                                         119895
E                                                         179808
F                                                          77005
G                                                         143874
H                                                          54491
I                                                         100800
K                                                         149550
L                        

## results of non detected CK accounting: 

* Total C = 40,115
* Total K = 149,550
* Sum in ukb fasta = 189,665
* sumUKB - dbNSFPnontarget = 14,963...which is 2 less than total number of target C and K...so only short 2 amino acids from total count!

In [133]:
print(149550 + 40115) # CK sum in ukb
print(189665 - 174702) # difference btw ukb and dbNSFP non detected CK positions
# 14963 , which is 2 less than it is suppose to be

189665
14963


In [147]:
# det_final_accounting = det_final[['matched_UKBID_x', 'pos_ID','Cys_reactivity', 'Cys_react_threshold', 'Cys_target_label', 'Lys_reactivity', 'Lys_react_threshold', 'Lys_target_label']]
det_final_accounting = det_final[['aaref_x', 'pos_ID']]
det_final_accounting.drop_duplicates(inplace=True)
det_final_accounting.head(3)

/Users/mariapalafox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,aaref_x,pos_ID
0,C,Q3ZCM7_C354
7,C,Q3ZCM7_C303
14,C,Q3ZCM7_C239


In [148]:
# everlabeled C or K count
checkColumnValues(det_final_accounting, 'aaref_x')

  aaref_x  Count
0       K   8868
1       C   6057


In [141]:
checkColumnValues(det_final_accounting, 'Cys_react_threshold')
print()
checkColumnValues(det_final_accounting, 'Cys_target_label')
print()
checkColumnValues(det_final_accounting, 'Lys_react_threshold')
print()
checkColumnValues(det_final_accounting, 'Lys_target_label')

  Cys_react_threshold  Count
0                 Low    895
1              Medium    412
2                High     94

  Cys_target_label  Count
0      panReactive   5088
1           Target    708

  Lys_react_threshold  Count
0                 Low   3401
1              Medium    660
2                High    302

  Lys_target_label  Count
0      panReactive   7840
1           Target    115


In [143]:
not_final_accounting = not_final[["aaref", "pos_ID_falseCKtarget"]]
not_final_accounting.drop_duplicates(inplace=True)
not_final_accounting.head(3)

/Users/mariapalafox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,aaref,pos_ID_falseCKtarget
0,K,Q3ZCM7_K392
7,K,Q3ZCM7_K379
14,K,Q3ZCM7_K362


In [144]:
checkColumnValues(not_final_accounting, 'aaref')

  aaref   Count
0     K  140652
1     C   34050


### FINAL ACCOUNTING of filtered/checked CK positions for .bed file:
* total uniprot IDs = 3840
* all targets found with 7 changes per codon
* only 2 CK non-targets missing and 2 aminos with incorrect codon change count

| TYPE | Cysteine | Lysine | 
|-------|--------------|-----------|
| Low | 895 | 3401 | 
| Medium | 412 | 660 |
| High | 94 | 302 |
| Panreactive | 5088 | 7804 |
| TargetSM | 708 | 115 |
| EVER-labeled | 6057 | 8868 |
| not-labeled | 34050 | 140652 |

# [MAKE BED FILES WITH hg19 COORDINATES]

In [152]:
sampledet25 = det_final[0:25]

In [153]:
sampledet25.to_csv("detected25_sample.csv")